# Title

In [ ]:
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'  # always print last expr.
%config InlineBackend.figure_format = 'svg'
%load_ext autoreload
%autoreload 2
%matplotlib inline

import logging

logging.basicConfig(level=logging.INFO)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from pandas import DataFrame, DatetimeIndex, Series

np.set_printoptions(precision=4, floatmode="fixed", suppress=True)
rng = np.random.default_rng()

In [ ]:
from tsdm.datasets import Electricity

ds = Electricity()

In [ ]:
dt = ds.index
time_resolution = [
    dt.year,
    dt.month,
    dt.day,
    dt.hour,
    dt.minute,
    dt.second,
    dt.microsecond,
    dt.nanosecond,
]

In [ ]:
import pandas as pd

pd.Timedelta(ds.index.inferred_freq)

In [ ]:
dt.to_series().inferred_freq

In [ ]:
class SocialTime:
    level_codes = {
        "Y": "year",
        "M": "month",
        "W": "weekday",
        "D": "day",
        "h": "hour",
        "m": "minute",
        "s": "second",
        "µ": "microsecond",
        "n": "nanosecond",
    }

    def __init__(self, levels: str = "YMWDhms") -> None:
        self.levels = [self.level_codes[k] for k in levels]

    def fit(self, x: Series, /) -> None:
        self.original_type = type(x)
        self.original_name = x.name
        self.original_dtype = x.dtype
        self.rev_cols = [l for l in self.levels if l != "weekday"]
        # self.new_names = {level:f"{x.name}_{level}" for level in self.levels}
        # self.rev_names = {f"{x.name}_{level}":level for level in self.levels if level != "weekday"}

    def encode(self, x, /):
        if isinstance(x, DatetimeIndex):
            res = {level: getattr(x, level) for level in self.levels}
        else:
            res = {level: getattr(x, level) for level in self.levels}
        return DataFrame.from_dict(res)

    def decode(self, x, /):
        x = x[self.rev_cols]
        s = pd.to_datetime(x)
        return self.original_type(s, name=self.original_name, dtype=self.original_dtype)

In [ ]:
enc.new_names

In [ ]:
enc = SocialTime()
enc.fit(ds.index)
encoded = enc.encode(ds.index)

In [ ]:
enc.decode(encoded)

In [ ]:
ds.index